# **14.2 Writing_Text_Files**

Reading files is only half the story — your Pokemon game needs to save progress, export team rosters, write logs, and generate reports. In this lesson you'll learn how to write data to files safely, the differences between write modes, how to append to existing files, and best practices for ensuring your data is saved correctly without corrupting important files.

---

## **Write Mode vs Append Mode**

When opening a file for writing, you must choose between **write mode** (`'w'`) which erases existing content, and **append mode** (`'a'`) which adds to the end. Understanding the difference prevents accidental data loss.

In [ ]:
# Mode 'w' — WRITE mode (creates new file or OVERWRITES existing)
with open('demo_write.txt', 'w') as f:
    f.write("First line\n")
    f.write("Second line\n")

# Read it back to verify
with open('demo_write.txt', 'r') as f:
    print("After first write:")
    print(f.read())

# Write again — this REPLACES all previous content!
with open('demo_write.txt', 'w') as f:
    f.write("This replaces everything\n")

with open('demo_write.txt', 'r') as f:
    print("After second write (content was erased):")
    print(f.read())

# Mode 'a' — APPEND mode (adds to end, never erases)
with open('demo_append.txt', 'w') as f:
    f.write("Initial content\n")

with open('demo_append.txt', 'a') as f:  # 'a' = append
    f.write("Line 2\n")
    f.write("Line 3\n")

with open('demo_append.txt', 'r') as f:
    print("\nAfter append (nothing was erased):")
    print(f.read())

---

## **Writing Strings with write()**

The `write()` method takes a string and writes it to the file. Unlike `print()`, it does **not** automatically add a newline — you must include `\n` yourself when you want line breaks.

In [ ]:
# write() does NOT add newlines automatically
with open('pokemon_team.txt', 'w') as f:
    f.write("Pikachu")      # No \n — stays on same line
    f.write("Charizard")    # Appends right after Pikachu

with open('pokemon_team.txt', 'r') as f:
    print("Without newlines:")
    print(repr(f.read()))  # Shows 'PikachuCharizard' — no line break

# Add \n manually for line breaks
with open('pokemon_team.txt', 'w') as f:
    f.write("Pikachu\n")     # \n creates a line break
    f.write("Charizard\n")
    f.write("Blastoise\n")

with open('pokemon_team.txt', 'r') as f:
    print("\nWith newlines:")
    print(f.read())

---

## **Writing Multiple Lines with writelines()**

The `writelines()` method takes a list of strings and writes them all at once. Like `write()`, it does **not** add newlines — if you want them, include `\n` in each string.

In [ ]:
# writelines() takes a list of strings
team = ["Pikachu", "Charizard", "Blastoise", "Venusaur"]

# WITHOUT \n — all on one line
with open('team_bad.txt', 'w') as f:
    f.writelines(team)  # Writes 'PikachuCharizardBlastoiseVenusaur'

with open('team_bad.txt', 'r') as f:
    print("Without newlines:")
    print(f.read())

# WITH \n — proper line breaks
team_with_newlines = [name + "\n" for name in team]

with open('team_good.txt', 'w') as f:
    f.writelines(team_with_newlines)

with open('team_good.txt', 'r') as f:
    print("\nWith newlines:")
    print(f.read())

---

## **Saving Pokemon Data Structures**

When saving lists or dictionaries, you need to convert them to strings first. The simplest approach for CSV-like data is to format each item as a line with commas (or tabs, or whatever delimiter you choose).

In [ ]:
# Save a list of Pokemon dictionaries to a CSV-like file
team = [
    {"name": "Pikachu",  "type": "Electric", "level": 25, "hp": 35},
    {"name": "Charizard","type": "Fire",     "level": 36, "hp": 78},
    {"name": "Blastoise","type": "Water",    "level": 36, "hp": 79},
    {"name": "Venusaur", "type": "Grass",    "level": 32, "hp": 80},
]

def save_pokemon_team(filename: str, team: list[dict]) -> None:
    """
    Save Pokemon team to a CSV file.
    Format: name,type,level,hp
    """
    with open(filename, 'w') as f:
        # Write header line
        f.write("name,type,level,hp\n")
        
        # Write each Pokemon as a line
        for pokemon in team:
            line = f"{pokemon['name']},{pokemon['type']},{pokemon['level']},{pokemon['hp']}\n"
            f.write(line)

save_pokemon_team('team_save.csv', team)
print("Team saved to team_save.csv")

# Read it back to verify
with open('team_save.csv', 'r') as f:
    print("\nFile contents:")
    print(f.read())

---

## **Appending to Log Files**

Append mode is perfect for log files where you want to add events over time without losing previous entries. Each new session adds to the end instead of wiping the history.

In [ ]:
import datetime

def log_battle(attacker: str, defender: str, damage: int) -> None:
    """
    Append a battle event to the log file with a timestamp.
    Each call adds a new line — previous entries are preserved.
    """
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {attacker} dealt {damage} damage to {defender}\n"
    
    with open('battle_log.txt', 'a') as f:  # 'a' = append
        f.write(log_entry)

# Simulate several battle events
log_battle("Pikachu",  "Onix",      45)
log_battle("Onix",     "Pikachu",   28)
log_battle("Pikachu",  "Onix",      52)

print("Battle events logged to battle_log.txt\n")

# Read the complete log
with open('battle_log.txt', 'r') as f:
    print("Battle Log:")
    print(f.read())

---

## **Safe Writing with Temporary Files**

When updating critical files (like save files), write to a temporary file first, then rename it. This prevents corruption if the program crashes mid-write — the original stays intact.

In [ ]:
import os

def save_game_safely(filename: str, data: str) -> None:
    """
    Safely save game data using a temporary file.
    If the write fails, the original file is not corrupted.
    """
    temp_filename = filename + ".tmp"
    
    # Write to temporary file first
    with open(temp_filename, 'w') as f:
        f.write(data)
    
    # Only replace the original after successful write
    os.replace(temp_filename, filename)  # Atomic on most systems
    print(f"Game safely saved to {filename}")

save_data = """Trainer: Ash
Badges: 8
Team: Pikachu,Charizard,Blastoise
"""

save_game_safely('savefile.txt', save_data)

# Verify
with open('savefile.txt', 'r') as f:
    print("\nSave file contents:")
    print(f.read())

---

## **Writing with Different Encodings**

Just like reading, you can specify an encoding when writing. UTF-8 handles all Unicode characters (including emoji and accented Pokemon names), so it's the recommended default.

In [ ]:
# Write a file with special characters using UTF-8
pokemon_names = [
    "Pikachu ⚡",
    "Flabébé 🌸",
    "Pokémon Trainer",
]

with open('special_chars.txt', 'w', encoding='utf-8') as f:
    for name in pokemon_names:
        f.write(name + "\n")

# Read it back
with open('special_chars.txt', 'r', encoding='utf-8') as f:
    print("File with special characters:")
    print(f.read())

---

## **Flushing and Buffering**

For performance, Python buffers writes to disk. Usually this is invisible, but if you're writing logs or debugging, you might want to force immediate writes with `flush()`.

In [ ]:
import time

# Demonstrate buffering and flushing
with open('progress.txt', 'w') as f:
    for i in range(5):
        f.write(f"Processing Pokemon {i+1}...\n")
        f.flush()  # Force write to disk immediately
        time.sleep(0.5)  # Simulate work
    
    # Without flush(), all writes might happen at once when the file closes

with open('progress.txt', 'r') as f:
    print("Progress log:")
    print(f.read())

---

## **Practical: Complete Save/Load System**

In [ ]:
# A complete Pokemon team save/load system

def save_team(filename: str, team: list[dict]) -> bool:
    """
    Save Pokemon team to a file.
    Returns True on success, False on failure.
    """
    try:
        with open(filename, 'w') as f:
            # Write header
            f.write("# Pokemon Team Save File\n")
            f.write("# Format: name|type|level|hp\n")
            f.write("\n")
            
            # Write each Pokemon
            for p in team:
                line = f"{p['name']}|{p['type']}|{p['level']}|{p['hp']}\n"
                f.write(line)
        
        print(f"✓ Team saved to {filename}")
        return True
    
    except Exception as e:
        print(f"✗ Save failed: {e}")
        return False

def load_team(filename: str) -> list[dict]:
    """
    Load Pokemon team from a file.
    Returns empty list if file doesn't exist or is invalid.
    """
    team = []
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                
                # Skip comments and empty lines
                if not line or line.startswith('#'):
                    continue
                
                # Parse data
                parts = line.split('|')
                if len(parts) != 4:
                    continue
                
                name, ptype, level_str, hp_str = parts
                
                team.append({
                    'name': name,
                    'type': ptype,
                    'level': int(level_str),
                    'hp': int(hp_str)
                })
        
        print(f"✓ Loaded {len(team)} Pokemon from {filename}")
    
    except FileNotFoundError:
        print(f"✗ File '{filename}' not found")
    except Exception as e:
        print(f"✗ Load failed: {e}")
    
    return team

# Test the system
original_team = [
    {"name": "Pikachu",   "type": "Electric", "level": 25, "hp": 35},
    {"name": "Charizard", "type": "Fire",     "level": 36, "hp": 78},
    {"name": "Blastoise", "type": "Water",    "level": 36, "hp": 79},
]

# Save
save_team('my_team.txt', original_team)

# Load back
loaded_team = load_team('my_team.txt')

print("\nLoaded team:")
for p in loaded_team:
    print(f"  {p['name']:12} ({p['type']:8}) Lv.{p['level']} HP:{p['hp']}")

---

## **Practice Exercises**

### **Task 1: Write a Simple File**

Write "Pikachu" to a file using write mode.

**Expected Output:**
```
File created
```

In [ ]:
# Your code here:


### **Task 2: Write Multiple Lines**

Write three Pokemon names, each on its own line.

**Expected Output (in file):**
```
Pikachu
Charizard
Blastoise
```

In [ ]:
# Your code here:


### **Task 3: Use writelines()**

Write a list of Pokemon names using `writelines()`, adding `\n` to each.

**Expected Output (in file):**
```
Pikachu
Charizard
Blastoise
```

In [ ]:
names = ["Pikachu", "Charizard", "Blastoise"]

# Your code here:


### **Task 4: Append to File**

Create a file with one Pokemon, then append two more.

**Expected Output (in file):**
```
Pikachu
Charizard
Blastoise
```

In [ ]:
# Your code here:


### **Task 5: Save CSV Data**

Write a list of Pokemon dicts to a CSV file with format `name,type,level`.

**Expected Output (in file):**
```
Pikachu,Electric,25
Charizard,Fire,36
```

In [ ]:
team = [
    {"name": "Pikachu",   "type": "Electric", "level": 25},
    {"name": "Charizard", "type": "Fire",     "level": 36},
]

# Your code here:


### **Task 6: Write with Header**

Write a CSV with a header row.

**Expected Output (in file):**
```
name,level
Pikachu,25
Charizard,36
```

In [ ]:
# Your code here:


### **Task 7: Log Events**

Write a function that appends battle events to a log file.

**Expected Output (in file after 2 calls):**
```
Pikachu used Thunderbolt
Charizard used Flamethrower
```

In [ ]:
# Your code here:


### **Task 8: Handle Write Errors**

Try to write to a file and catch any exceptions.

**Expected Output:**
```
File saved successfully
(or error message if it fails)
```

In [ ]:
# Your code here:


### **Task 9: Overwrite Warning**

Check if a file exists before writing, warn the user if it will be overwritten.

**Expected Output:**
```
Warning: file exists and will be overwritten
```

In [ ]:
import os

# Your code here:


### **Task 10: Round-trip Test**

Save a team to a file, then load it back and verify it matches.

**Expected Output:**
```
Save/load successful: data matches
```

In [ ]:
# Your code here:


---

## **Summary**

- `open(file, 'w')` — write mode (creates new or **overwrites** existing)
- `open(file, 'a')` — append mode (adds to end, preserves content)
- `write(string)` — write one string (no auto-newline)
- `writelines(list)` — write list of strings (no auto-newlines)
- Always include `\n` manually when you want line breaks
- Use `with` to auto-close files
- `flush()` forces immediate write to disk
- Specify `encoding='utf-8'` for special characters
- Safe pattern: write to temp file, then rename
- Append mode is perfect for logs

---

## **Quick Reference**

```python
# Write (overwrites)
with open('file.txt', 'w') as f:
    f.write("Line 1\n")
    f.write("Line 2\n")

# Append (adds to end)
with open('file.txt', 'a') as f:
    f.write("Line 3\n")

# Write multiple lines
lines = ["A\n", "B\n", "C\n"]
with open('file.txt', 'w') as f:
    f.writelines(lines)

# Safe save pattern
temp = filename + ".tmp"
with open(temp, 'w') as f:
    f.write(data)
os.replace(temp, filename)

# With encoding
with open('file.txt', 'w', encoding='utf-8') as f:
    f.write("Pokémon ⚡\n")
```